# Hands on : Deterministic forecasting

## Lorenz 96 Model

The Lorenz 96 model is a simplified dynamical system used to represent the atmosphere and study chaos in weather and climate dynamics. It consists of a set of coupled variables that evolve over time, with each variable representing different aspects of the system (e.g., temperature or pressure). The model is widely used to investigate stochastic parametrizations and the effects of different levels of resolution in weather and climate models.

### Model Overview

The Lorenz 96 model defines two types of variables:
- **Slow Variables $X_k$**: These are large-amplitude, slow-changing variables, which represent the 'resolved' components of the system.
- **Fast Variables $Y_j$**: These are small-amplitude, fast-changing variables, representing the 'unresolved' components.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

from l96 import *
from tqdm import tqdm
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score 
from utils import *
np.random.seed(3)

# Define colorblind-friendly colors
color_sim = '#56B4E9'   # Light blue
color_threshold = '#D55E00'  # Red

In [ ]:
initX, initY = np.load('./data/initX.npy'), np.load('./data/initY.npy')
l96_ref = L96TwoLevel(X_init=initX, Y_init=initY, save_dt=0.001, noYhist=False)
l96_ref.iterate(30)
l96_ref.erase_history()
l96_ref.iterate(0.8)

# Create polar animation for Lorenz 96 simulation
def plot_l96_polar_animation(X_hist, Y_hist, subsample_rate=10):
    K = X_hist.shape[1]  # Number of points for X
    J = Y_hist.shape[1]  # Number of points for Y
    
    # Polar coordinates for plotting
    theta = np.linspace(0, 2 * np.pi, K, endpoint=False)
    theta = np.append(theta, theta[0])  # Ensure circular loop
    theta_Y = np.linspace(0, 2 * np.pi, J, endpoint=False)
    theta_Y = np.append(theta_Y, theta_Y[0])  # Ensure circular loop
    
    # Subsample for faster animation
    subsample_time = range(0, len(X_hist), subsample_rate)
    X_hist_sub = X_hist[subsample_time]
    Y_hist_sub = Y_hist[subsample_time]
    
    # Create polar plot
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    ax.set_ylim(-15, 15)  # Adjust radial limits
    
    # Initialize lines for X and Y
    line_X, = ax.plot([], [], 'b-', label='X')
    line_Y, = ax.plot([], [], 'orange', alpha=0.7, label='Y')
    
    # Initialize function for animation
    def init():
        line_X.set_data([], [])
        line_Y.set_data([], [])
        return line_X, line_Y
    
    # Update function for animation
    def update(frame):
        X_values = np.append(X_hist_sub[frame], X_hist_sub[frame][0])  # Loop back to start
        Y_values = np.append(Y_hist_sub[frame], Y_hist_sub[frame][0])  # Loop back to start
        
        line_X.set_data(theta, X_values)
        line_Y.set_data(theta_Y, Y_values)
        return line_X, line_Y
    
    # Create animation
    ani = animation.FuncAnimation(fig, update, frames=len(X_hist_sub), init_func=init, blit=True, interval=50)
    
    # Close static plot to avoid double display
    plt.close(fig)
    
    # Display animation in Jupyter Notebook
    return HTML(ani.to_jshtml())

# Display the polar animation directly in the Jupyter Notebook
plot_l96_polar_animation(l96_ref.history.X.values, l96_ref.history.Y.values*10)


#### Equations

The evolution of the slow and fast variables are given by:

$$
\frac{dX_k}{dt} = -X_{k-1} (X_{k-2} - X_{k+1}) - X_k + F - \frac{h}{b} \bar{Y} \quad \text{for} \quad k = 1, \dots, K
$$

$$
\frac{dY_j}{dt} = -c b Y_{j+1} (Y_{j+2} - Y_{j-1}) - c Y_j + \frac{h}{b} X_{k}  \quad \text{for} \quad j = 1, \dots, JK
$$

Where:
- $X_k$ are the slow variables.
- $Y_j$ are the fast variables.
- $F$ is the external forcing term.
- $h$, $b$, and $c$ are model parameters.
- $X_{\text{int}}$ represents the interaction of slow components with fast components.

### Parameters

The model uses the following parameters, which control the dynamics of the system:

| Parameter | Description                          | Value     |
|-----------|--------------------------------------|-----------|
| $K$   | Number of slow variables             | 8         |
| $J$   | Number of fast variables per slow var| 32        |
| $h$   | Coupling constant for interaction    | 1         |
| $b$   | Scaling constant for fast components | 10        |
| $c$   | Scaling constant for fast components | 10        |
| $F$   | External forcing term                | 18 or 20  |
| $X_{\text{int}}$ | Interaction term for coupling between layers | Defined by the integer truncation $\left\lfloor \frac{j-1}{J} \right\rfloor + 1$ |


In [ ]:
l96_ref.erase_history()
l96_ref.iterate(15)
h = l96_ref.history
X = h.X.values
idx = np.arange(0, X.shape[0], 100)
X_sampled = X[idx, :]
observations = X_sampled + np.random.normal(0, 0.3,  X_sampled.shape)
threshold = np.percentile(observations, 99.9)

### Goal
* We want to accurately predict future storms which we set as the 99.9 percentile on the past distribution (approximately $X>12$) 
* We are interested in the Southeastern of UK which is located at $X[:, 0]$.

In [ ]:
location = 0

In [ ]:
plt.plot(np.arange(0, X.shape[0]), X[:, location], label='True value')
plt.plot(idx, observations[:, location], 'x', color='darkblue', label='observation')
plt.hlines(y=threshold, xmin=0, xmax=idx[-1], linestyles='-.', label='Storm threshold', color='black')

## Simulation Model

The simulation model we use is designed to forecast the dynamics of the resolved variables $X_k^*$ by approximating the effects of the unresolved variables $Y_j$. Instead of explicitly modeling the fast components, their influence is captured through a parametrization function $g_U(X_k^*)$, which depends solely on the resolved variables. This approach simplifies the system while retaining the key characteristics of the unresolved dynamics. The model is governed by the following equation:

$$
\frac{dX_k^*}{dt} = -X_{k-1}^*(X_{k-2}^* - X_{k+1}^*) - X_k^* + F - g_U(X_k^*) \quad \text{for} \quad k = 1, \dots, K
$$

In this formulation, the term $g_U(X_k^*)$ models the impact of the unresolved variables and includes random forcing, reflecting the inherent uncertainty in the system. This simplified representation enables efficient simulations of the resolved variables while accounting for the unresolved dynamics through parametrization.


### Running a simple simulation model
We initialise our simulation model from the last observation:

In [ ]:
X_init = observations[-1, :]

Our simple deterministic simulation model works as follows:
* We firt load the simulation model

In [ ]:
l96_simulation = L96OneLevel(X_init=X_init, noprog=True)

* Using the function [np.roll](https://numpy.org/doc/2.1/reference/generated/numpy.roll.html) create a function that take $X_k \in \mathbb{R}^K$ and return for all $k$, $\frac{dX_k}{d t} = -X_{k-1} (X_{k-2} - X_{k+1}) - X_k + F$ with $F=20$ (assumed as known).

In [ ]:
def _rhs(X, F=10):
    """Right the upating function here using function np.roll."""
    dXdt = (
            # Fill the dynamical system here 
    )
    return dXdt

l96_simulation._rhs = _rhs

In [ ]:
time = 10
l96_simulation.iterate(time=time)

l96_ref.erase_history()
l96_ref.iterate(time=time)

**Asking people what would be there communication about what is gonna happen**

In [ ]:
h_sim = l96_simulation.history
h_true = l96_ref.history

plot_time_series(X, h_true.X.values, h_sim.X.values[None, :, :], subsample_rate=50, threshold=threshold)

![](../content/images/great_storm_news_paper.jpeg)

The [Great Storm of 1987](https://en.wikipedia.org/wiki/Great_storm_of_1987) hit southeastern Britain unexpectedly, causing widespread damage. Though there were some early warnings, the storm was not predicted for the night it struck, catching people by surprise. Winds reached up to **220 km/h**, wreaking havoc across southern England. Thousands of trees were uprooted, and parks, estates, and landmarks were severely damaged. Transportation was crippled as roads, railways, and airports were closed, while widespread power outages plunged much of the southeast, including London, into darkness. The storm resulted in **18 fatalities** and left many homes and schools in ruins. It also caused **£2 billion in damage**, making it the second most costly weather event in UK history. Though the storm’s timing at night helped reduce casualties, it highlighted the significant challenges of weather forecasting.

![](../content/images/great_storm_news_paper_2.png)

In the early 1990s, following the unexpected storm of 1987, weather forecasting began shifting from simple deterministic predictions to **probabilistic ensemble forecasting**. This approach runs multiple simulations with slightly different initial conditions, providing a range of possible outcomes rather than a single forecast. Advances in computing power and parallel processing made this transition possible, allowing meteorologists to better account for uncertainties in atmospheric conditions. Ensemble forecasting has since become a key tool in operational weather prediction, improving the ability to anticipate extreme events and their likelihood.

See [Lessons and legacy of the Great Storm of 1987 (MetOffice)](https://www.metoffice.gov.uk/about-us/who-we-are/our-history/lessons-and-legacy-of-the-great-storm-of-1987) for more details on how the great storm of 1987 helped in improving our modelling capacities.

See also [The great storm of 15–16 October 1987](https://www.researchgate.net/profile/Stephen-Burt/publication/260891819_The_Great_Storm_of_15-16_October_1987/links/5aec4f380f7e9b01d3e08884/The-Great-Storm-of-15-16-October-1987.pdf).

### Model performance
* Let's make our failure more quantitative
* Plot the time evolution of the MSE between observations and prediction

In [ ]:
observations = h_true.X.values
predictions = h_sim.X.values[1:, :]

In [ ]:
# Plot MSE over time

### Questions
* Why did we fail capturing this extreme event?
* How could avoid this problem in the future?